Update the password defined here

https://github.com/sebastian-luna-valero/pangeo-eosc/blob/egi/EGI.md#step-2-deploy-pangeo


In [1]:
from dask_gateway import Gateway
from dask_gateway.auth import BasicAuth
auth = BasicAuth(password="xxxxx")
gateway = gateway = Gateway(
    "http://api-daskhub-dask-gateway.daskhub:8000/",
    auth=auth,
)

In [2]:
cluster = gateway.new_cluster()

In [3]:
cluster

In [4]:
options = gateway.cluster_options()
options

In [5]:
options

In [6]:
from distributed import Client
client=Client(cluster)

/srv/conda/envs/notebook/lib/python3.9/site-packages/distributed/client.py:1274: VersionMismatchWarning: Mismatched versions found

+---------+----------------+----------------+----------------+
| Package | client         | scheduler      | workers        |
+---------+----------------+----------------+----------------+
| lz4     | 4.0.0          | None           | None           |
| pandas  | 1.4.2          | None           | None           |
| python  | 3.9.13.final.0 | 3.10.5.final.0 | 3.10.5.final.0 |
+---------+----------------+----------------+----------------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


In [7]:
client

Connection method: Cluster object,Cluster type: dask_gateway.GatewayCluster
Dashboard: /services/dask-gateway/clusters/daskhub.7ec35ddb400a4d3a83d4910cf32964f4/status,


In [8]:
import xarray as xr

In [9]:
ds=xr.tutorial.open_dataset("air_temperature",chunks={"time":500})

In [10]:
ds=ds.persist()

In [11]:
client.get_versions(check=True)

/srv/conda/envs/notebook/lib/python3.9/site-packages/distributed/client.py:4239: UserWarning: Mismatched versions found

+---------+----------------+----------------+----------------+
| Package | client         | scheduler      | workers        |
+---------+----------------+----------------+----------------+
| lz4     | 4.0.0          | None           | None           |
| pandas  | 1.4.2          | None           | None           |
| python  | 3.9.13.final.0 | 3.10.5.final.0 | 3.10.5.final.0 |
+---------+----------------+----------------+----------------+
  warnings.warn(msg["warning"])


{'scheduler': {'host': {'python': '3.10.5.final.0',
   'python-bits': 64,
   'OS': 'Linux',
   'OS-release': '5.4.0-109-generic',
   'machine': 'x86_64',
   'processor': '',
   'byteorder': 'little',
   'LC_ALL': 'None',
   'LANG': 'C.UTF-8'},
  'packages': {'python': '3.10.5.final.0',
   'dask': '2022.6.0',
   'distributed': '2022.6.0',
   'msgpack': '1.0.4',
   'cloudpickle': '2.1.0',
   'tornado': '6.1',
   'toolz': '0.11.2',
   'numpy': '1.22.4',
   'pandas': None,
   'lz4': None}},
 'workers': {'tls://10.244.1.16:34183': {'host': {'python': '3.10.5.final.0',
    'python-bits': 64,
    'OS': 'Linux',
    'OS-release': '5.4.0-109-generic',
    'machine': 'x86_64',
    'processor': '',
    'byteorder': 'little',
    'LC_ALL': 'None',
    'LANG': 'C.UTF-8'},
   'packages': {'python': '3.10.5.final.0',
    'dask': '2022.6.0',
    'distributed': '2022.6.0',
    'msgpack': '1.0.4',
    'cloudpickle': '2.1.0',
    'tornado': '6.1',
    'toolz': '0.11.2',
    'numpy': '1.22.4',
    'pandas

In [12]:
from dask import delayed
import numpy as np
f=delayed(lambda x: np.exp(x*x))

#client.submit(f(2))
client.compute(f(2))



<Future: pending, key: lambda-0f25f7e1-f906-4d79-b995-8aa11c090554>

In [13]:
import dask
dask.compute(f(2))

(54.598150033144236,)

In [14]:
from distributed.diagnostics.plugin import PipInstall

In [15]:
extra_packages=["xarray","netCDF4","pooch","h5netcdf"]
plugin=PipInstall(extra_packages,restart=True)

In [19]:
client.register_worker_plugin(plugin)

{'tls://10.244.1.16:34183': {'status': 'OK'}}

In [20]:
client.get_versions(packages=extra_packages,check=True)

/srv/conda/envs/notebook/lib/python3.9/site-packages/distributed/client.py:4239: UserWarning: Mismatched versions found

+---------+--------+-----------+---------+
| Package | client | scheduler | workers |
+---------+--------+-----------+---------+
| lz4     | 4.0.0  | None      | None    |
+---------+--------+-----------+---------+
  warnings.warn(msg["warning"])


{'scheduler': {'host': {'python': '3.10.5.final.0',
   'python-bits': 64,
   'OS': 'Linux',
   'OS-release': '5.4.0-109-generic',
   'machine': 'x86_64',
   'processor': '',
   'byteorder': 'little',
   'LC_ALL': 'None',
   'LANG': 'C.UTF-8'},
  'packages': {'python': '3.10.5.final.0',
   'dask': '2022.6.0',
   'distributed': '2022.6.0',
   'msgpack': '1.0.4',
   'cloudpickle': '2.1.0',
   'tornado': '6.1',
   'toolz': '0.11.2',
   'numpy': '1.22.4',
   'pandas': None,
   'lz4': None,
   'xarray': None,
   'netCDF4': None,
   'pooch': None,
   'h5netcdf': None}},
 'workers': {},
 'client': {'host': {'python': '3.9.13.final.0',
   'python-bits': 64,
   'OS': 'Linux',
   'OS-release': '5.4.0-109-generic',
   'machine': 'x86_64',
   'processor': 'x86_64',
   'byteorder': 'little',
   'LC_ALL': 'C.UTF-8',
   'LANG': 'C.UTF-8'},
  'packages': {'python': '3.9.13.final.0',
   'dask': '2022.6.0',
   'distributed': '2022.6.0',
   'msgpack': '1.0.4',
   'cloudpickle': '2.1.0',
   'tornado': '6.1

In [25]:
ds.air.mean(dim='time').persist()#.compute()

<xarray.DataArray 'air' (lat: 25, lon: 53)>
dask.array<mean_agg-aggregate, shape=(25, 53), dtype=float32, chunksize=(25, 53), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float32 75.0 72.5 70.0 67.5 65.0 ... 25.0 22.5 20.0 17.5 15.0
  * lon      (lon) float32 200.0 202.5 205.0 207.5 ... 322.5 325.0 327.5 330.0

In [26]:
url='https://github.com/pydata/xarray-data/blob/master/air_temperature.nc?raw=true'
ds=xr.open_dataset(url,engine="netcdf4")
#ds.compute()#.mean(dim="time").compute()#.air.data.visualize()
#ds=ds.chunk({"time":500})


OSError: [Errno -75] NetCDF: Malformed or unexpected Constraint: b'https://github.com/pydata/xarray-data/blob/master/air_temperature.nc?raw=true'

In [28]:
ds.mean(dim="time")#.compute()#.air.data.visualize()



<xarray.Dataset>
Dimensions:  (lat: 25, lon: 53)
Coordinates:
  * lat      (lat) float32 75.0 72.5 70.0 67.5 65.0 ... 25.0 22.5 20.0 17.5 15.0
  * lon      (lon) float32 200.0 202.5 205.0 207.5 ... 322.5 325.0 327.5 330.0
Data variables:
    air      (lat, lon) float32 dask.array<chunksize=(25, 53), meta=np.ndarray>